## 기본 예시: 프롬프트 + 모델 + 출력 파서

가장 기본적이고 일반적인 사용 사례는 prompt 템플릿과 모델을 함께 연결하는 것입니다. 이것이 어떻게 작동하는지 보기 위해, 각 나라별 수도를 물어보는 Chain을 생성해 보겠습니다.


In [4]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()

True

In [5]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install -qU langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("CH01-Basic")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH01-Basic


## 프롬프트 템플릿의 활용

`PromptTemplate`

- 사용자의 입력 변수를 사용하여 완전한 프롬프트 문자열을 만드는 데 사용되는 템플릿입니다
- 사용법
  - `template`: 템플릿 문자열입니다. 이 문자열 내에서 중괄호 `{}`는 변수를 나타냅니다.
  - `input_variables`: 중괄호 안에 들어갈 변수의 이름을 리스트로 정의합니다.

`input_variables`

- input_variables는 PromptTemplate에서 사용되는 변수의 이름을 정의하는 리스트입니다.

In [6]:
from langchain_teddynote.messages import stream_response  # 스트리밍 출력
from langchain_core.prompts import PromptTemplate

`from_template()` 메소드를 사용하여 PromptTemplate 객체 생성


In [9]:
# template 정의
template = "{country}의 수도는 어디인가요?"

# from_template 메소드를 이용하여 PromptTemplate 객체 생성
prompt_template = PromptTemplate.from_template(template)
prompt_template

PromptTemplate(input_variables=['country'], input_types={}, partial_variables={}, template='{country}의 수도는 어디인가요?')

In [10]:
# prompt 생성
prompt = prompt_template.format(country="대한민국")
prompt

'대한민국의 수도는 어디인가요?'

In [11]:
# prompt 생성
prompt = prompt_template.format(country="미국")
prompt

'미국의 수도는 어디인가요?'

In [10]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(
    model="gpt-4.1-nano",
    temperature=0.1,
)

## Chain 생성

### LCEL(LangChain Expression Language)

![lcel.png](./images/lcel.png)

여기서 우리는 LCEL을 사용하여 다양한 구성 요소를 단일 체인으로 결합합니다

```
chain = prompt | model | output_parser
```

`|` 기호는 [unix 파이프 연산자](<https://en.wikipedia.org/wiki/Pipeline_(Unix)>)와 유사하며, 서로 다른 구성 요소를 연결하고 한 구성 요소의 출력을 다음 구성 요소의 입력으로 전달합니다.

이 체인에서 사용자 입력은 프롬프트 템플릿으로 전달되고, 그런 다음 프롬프트 템플릿 출력은 모델로 전달됩니다. 각 구성 요소를 개별적으로 살펴보면 무슨 일이 일어나고 있는지 이해할 수 있습니다.


In [11]:
# prompt 를 PromptTemplate 객체로 생성합니다.
prompt = PromptTemplate.from_template("{topic} 에 대해 자세하게 설명해주세요.")

model = ChatOpenAI(model="gpt-4.1-nano", temperature=0.1)

chain = prompt | model

In [19]:
chain

PromptTemplate(input_variables=['topic'], input_types={}, partial_variables={}, template='{topic} 에 대해 자세하게 설명해주세요.')
| ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x12be39110>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x12c21e9d0>, root_client=<openai.OpenAI object at 0x12be36010>, root_async_client=<openai.AsyncOpenAI object at 0x12b598390>, model_name='gpt-4.1-nano', temperature=0.1, model_kwargs={}, openai_api_key=SecretStr('**********'))

### invoke() 호출

- python 딕셔너리 형태로 입력값을 전달합니다.(키: 값)
- invoke() 함수 호출 시, 입력값을 전달합니다.

In [20]:
# input 딕셔너리에 주제를 '인공지능 모델의 학습 원리'으로 설정합니다.
input = {"topic": "인공지능 모델의 학습 원리"}

In [17]:
# prompt 객체와 model 객체를 파이프(|) 연산자로 연결하고 invoke 메서드를 사용하여 input을 전달합니다.
# 이를 통해 AI 모델이 생성한 메시지를 반환합니다.
chain.invoke(input)

AIMessage(content="인공지능 모델의 학습 원리는 데이터를 이용해 모델이 문제를 해결하는 방법을 배우는 과정입니다. 일반적으로 다음과 같은 단계로 이루어집니다:\n\n1. **데이터 수집:** 모델이 학습할 수 있도록 많은 예제 데이터를 준비합니다.\n2. **모델 설계:** 문제에 맞는 인공지능 구조(예: 신경망)를 만듭니다.\n3. **학습 과정:** 데이터를 모델에 입력하고, 모델이 내린 예측과 실제 정답을 비교하여 오차를 계산합니다.\n4. **오차 줄이기:** 오차를 최소화하도록 모델의 내부 파라미터(가중치)를 조정하는 과정을 반복합니다. 이때 주로 '경사 하강법'이라는 방법을 사용합니다.\n5. **검증 및 개선:** 학습이 잘 되는지 검증 데이터를 통해 확인하고, 필요하면 모델을 개선합니다.\n\n이 과정을 통해 인공지능은 주어진 문제에 대해 더 정확하게 예측하거나 판단할 수 있게 됩니다.", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 224, 'prompt_tokens': 24, 'total_tokens': 248, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-nano-2025-04-14', 'system_fingerprint': 'fp_04d3664870', 'id': 'chatcmpl-CLAchuVldNgpvVlvbI8F3VLv5r7P6', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs'

아래는 스트리밍을 출력하는 예시 입니다.

In [15]:
# 스트리밍 출력을 위한 요청
answer = chain.stream(input)
# 스트리밍 출력
stream_response(answer)

물론입니다! 질문하신 내용은 파이썬의 IPython 또는 Jupyter Notebook 환경에서 발생하는 특정 객체와 관련된 것으로 보입니다. 구체적으로는:

```python
<bound method Kernel.raw_input of <ipykernel.ipkernel.IPythonKernel object at 0x11a1ae950>>
```

이 표현은 파이썬 객체의 문자열 표현으로, 두 가지 중요한 정보를 담고 있습니다:

1. **`<bound method Kernel.raw_input of ...>`**  
   - `bound method`는 특정 객체에 바인딩된(연결된) 메서드를 의미합니다.  
   - 즉, `Kernel.raw_input`라는 메서드가 `Kernel` 객체에 연결되어 있다는 의미입니다.  
   - 이 메서드는 `Kernel` 클래스 또는 관련 클래스에 정의된 `raw_input`이라는 함수입니다.

2. **`<ipykernel.ipkernel.IPythonKernel object at 0x11a1ae950>`**  
   - 이 부분은 `IPythonKernel` 클래스의 인스턴스 객체를 나타내며, 메모리 주소 `0x11a1ae950`에 위치해 있음을 보여줍니다.  
   - 즉, 이 객체는 Jupyter Notebook 또는 IPython 환경에서 커널을 담당하는 객체입니다.

---

### 좀 더 구체적으로 설명하면:

- **`IPythonKernel` 객체**는 Jupyter Notebook의 커널을 구현하는 핵심 객체입니다. 이 객체는 사용자 코드 실행, 입력/출력 처리, 커널과 클라이언트 간 통신 등을 담당합니다.

- **`raw_input` 메서드**는 사용자로부터 입력을 받기 위한 함수입니다. 과거 파이썬 2에서는 `raw_input()`이 표준 입력을 받는 함수였고, 파이썬 3에서는 `input()`으로 변경되었지만, 내부 구현이나 커널에서는 여전히 `raw_input`이라는 이름을 사용할 수 있습니다.

### 출력파서(Output Parser)


In [7]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

Chain 에 출력파서를 추가합니다.

In [12]:
# 프롬프트, 모델, 출력 파서를 연결하여 처리 체인을 구성합니다.
chain = prompt | model | output_parser

In [23]:
# chain 객체의 invoke 메서드를 사용하여 input을 전달합니다.
input = {"topic": "인공지능 모델의 학습 원리"}
chain.invoke(input)

'인공지능(AI) 모델의 학습 원리는 데이터를 이용하여 모델이 특정 작업을 수행할 수 있도록 내부 매개변수(가중치와 편향 등)를 조정하는 과정입니다. 이 과정을 좀 더 구체적으로 설명하면 다음과 같습니다.\n\n1. **데이터 수집과 준비**  \n   - 모델이 학습할 수 있도록 다양한 예제(데이터)를 수집합니다.  \n   - 데이터는 입력과 정답(레이블)으로 구성되어 있으며, 이를 전처리(정규화, 정제, 증강 등)하여 모델이 학습하기 적합한 형태로 만듭니다.\n\n2. **모델 구조 설계**  \n   - 인공신경망, 결정트리, 서포트 벡터 머신 등 다양한 알고리즘 중에서 문제에 적합한 모델 구조를 선택하거나 설계합니다.  \n   - 예를 들어, 딥러닝에서는 여러 층으로 구성된 신경망 구조를 사용합니다.\n\n3. **초기화**  \n   - 모델의 가중치와 편향을 무작위 또는 특정 규칙에 따라 초기화합니다.\n\n4. **순전파(Forward Propagation)**  \n   - 입력 데이터를 모델에 넣으면, 각 층을 거치면서 계산이 진행됩니다.  \n   - 이 과정에서 입력이 가중치와 곱해지고, 활성화 함수(예: ReLU, 시그모이드 등)를 통과하여 출력값(예측값)이 생성됩니다.\n\n5. **손실 함수 계산**  \n   - 모델의 예측값과 실제 정답 간의 차이를 손실 함수(loss function)를 통해 계산합니다.  \n   - 예를 들어, 평균제곱오차(MSE), 교차 엔트로피(cross-entropy) 등이 사용됩니다.\n\n6. **역전파(Backpropagation)**  \n   - 손실 값을 기준으로, 가중치와 편향이 얼마나 잘못 조정되어야 하는지 계산합니다.  \n   - 체인 룰(Chain Rule)을 이용하여 손실 함수의 기울기(gradient)를 각 가중치에 대해 계산합니다.\n\n7. **가중치 업데이트**  \n   - 계산된 기울기를 이용하여 가중치를 조정합니다.  \n   - 일반적으로 경사 하강법(Gradient

In [13]:
# 스트리밍 출력을 위한 요청
answer = chain.stream(input)
# 스트리밍 출력
stream_response(answer)

물론입니다! 질문하신 내용은 파이썬의 IPython 또는 Jupyter Notebook 환경에서 발생하는 특정 객체와 관련된 것으로 보입니다. 구체적으로는:

```python
<bound method Kernel.raw_input of <ipykernel.ipkernel.IPythonKernel object at 0x11a1ae950>>
```

이 표현은 파이썬 객체의 문자열 표현으로, 해당 객체의 어떤 속성이나 메서드에 대한 정보를 보여줍니다. 이를 하나씩 분석해보겠습니다.

---

### 1. 전체 구조 분석

- `<bound method Kernel.raw_input of <ipykernel.ipkernel.IPythonKernel object at 0x11a1ae950>>`

이 구조는 두 부분으로 나뉩니다:

1. **`<bound method Kernel.raw_input of ...>`**  
   - 이는 '바인딩된 메서드(bound method)'를 의미합니다.
   - 즉, `Kernel` 클래스의 `raw_input` 메서드가 특정 인스턴스에 바인딩되어 있다는 의미입니다.
   
2. **`<ipykernel.ipkernel.IPythonKernel object at 0x11a1ae950>`**  
   - 이 부분은 `IPythonKernel` 클래스의 인스턴스 객체를 나타내며, 메모리 주소 `0x11a1ae950`에 위치한 객체입니다.

---

### 2. 상세 설명

#### a. `IPythonKernel` 객체

- `ipykernel.ipkernel.IPythonKernel`은 Jupyter Notebook 또는 IPython 환경에서 사용하는 커널의 핵심 클래스입니다.
- 이 객체는 사용자 코드 실행, 셀 실행, 입력 요청 등 커널의 핵심 기능을 담당합니다.
- 이 객체는 여러 메서드와 속성을 가지고 있으며, 그중 하나가 `raw_input`입니다.

#### b. `raw_input` 메서드

- `raw_input`

### 템플릿을 변경하여 적용

- 아래의 프롬프트 내용을 얼마든지 **변경** 하여 테스트 해볼 수 있습니다.
- `model_name` 역시 변경하여 테스트가 가능합니다.

In [17]:
template = """
당신은 영어를 가르치는 10년차 영어 선생님입니다. 주어진 상황에 맞는 영어 회화를 작성해 주세요.
양식은 [FORMAT]을 참고하여 작성해 주세요.

#상황:
{question}

#FORMAT:
- 영어 회화:
- 한글 해석:
"""

# 프롬프트 템플릿을 이용하여 프롬프트를 생성합니다.
prompt = PromptTemplate.from_template(template)

# ChatOpenAI 챗모델을 초기화합니다.
model = ChatOpenAI(model_name="gpt-4.1-nano")

# 문자열 출력 파서를 초기화합니다.
output_parser = StrOutputParser()

In [18]:
# 체인을 구성합니다.
chain = prompt | model | output_parser

In [19]:
# 완성된 Chain을 실행하여 답변을 얻습니다.
print(chain.invoke({"question": "저는 식당에 가서 음식을 주문하고 싶어요"}))

- 영어 회화:  
"Hello! I’d like to order a table for two, please."  
"Could I see the menu, please?"  
"I’d like the grilled chicken with rice, please."  
"Can I get that without onions?"  
"Thank you. Could I also get a glass of water?"  

- 한글 해석:  
"안녕하세요! 두 사람 자리로 예약하고 싶은데요."  
"메뉴 좀 보여주시겠어요?"  
"그릴 치킨과 밥을 주문할게요."  
"양파 빼고 부탁드릴 수 있을까요?"  
"감사합니다. 물 한 잔도 주세요."


In [ ]:
# 완성된 Chain을 실행하여 답변을 얻습니다.
# 스트리밍 출력을 위한 요청
answer = chain.stream({"question": "저는 식당에 가서 음식을 주문하고 싶어요"})
# 스트리밍 출력
stream_response(answer)

In [20]:
# 이번에는 question 을 '미국에서 피자 주문'으로 설정하여 실행합니다.
# 스트리밍 출력을 위한 요청
answer = chain.stream({"question": "미국에서 피자 주문"})
# 스트리밍 출력
stream_response(answer)

- 영어 회화:  
"Hello! I’d like to order a large pepperoni pizza, please."  
"Can I also get some extra cheese on that?"  
"How long will the delivery take?"  
"Thank you! I’ll be here waiting."

- 한글 해석:  
"안녕하세요! 대형 페퍼로니 피자를 주문하고 싶은데요."  
"여기에 치즈를 더 넣어주실 수 있나요?"  
"배달이 얼마나 걸릴까요?"  
"고맙습니다! 여기서 기다릴게요."